In [1]:
import pandas as pd
import datetime as dt
import os
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_auth
import plotly.graph_objects as go
import tab1
import tab2



class db:
    def __init__(self):
        self.transactions = db.transation_init()
        self.cc = pd.read_csv(r'/Users/mac/Desktop/py/MOD_11/MOD_11_PROJEKT/MOD_11_sketches_ProjectDash/db/country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'/Users/mac/Desktop/py/MOD_11/MOD_11_PROJEKT/MOD_11_sketches_ProjectDash/db/customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'/Users/mac/Desktop/py/MOD_11/MOD_11_PROJEKT/MOD_11_sketches_ProjectDash/db/prod_cat_info.csv')


    @staticmethod
    def transation_init():
        transactions = pd.DataFrame()
        src = r'/Users/mac/Desktop/py/MOD_11/MOD_11_PROJEKT/MOD_11_sketches_ProjectDash/db/transactions'
        for filename in os.listdir(src):
            transactions = pd.concat([transactions, pd.read_csv(os.path.join(src,filename),index_col=0)])

        def convert_dates(x):
            try:
                return dt.datetime.strptime(x,'%d-%m-%Y')
            except:
                return dt.datetime.strptime(x,'%d/%m/%Y')

        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

        return transactions


    def merge(self):
        df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
        .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

        df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
        .set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')

        df = df.join(self.customers.join(self.cc,on='country_code')
        .set_index('customer_Id'),on='cust_id')

        self.merged = df


df = db()
df.merge()


In [12]:
df['weekday'] = df['tran_date'].dt.dayofweek
df



,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,prod_cat,prod_subcat,DOB,Gender,country_code,country,weekday
0,80712190438,270351,2019-02-28,1,1,-5,-772,405.300,-4265.300,e-Shop,Clothing,Women,26-09-1981,M,5.0,Italy,3
1,29258453508,270384,2019-02-27,5,3,-5,-1497,785.925,-8270.925,e-Shop,Electronics,Computers,11-05-1973,F,8.0,Denmark,2
2,51750724947,273420,2019-02-24,6,5,-2,-791,166.110,-1748.110,TeleShop,Books,DIY,27-07-1992,M,8.0,Denmark,6
3,93274880719,271509,2019-02-24,11,6,-3,-1363,429.345,-4518.345,e-Shop,Home and kitchen,Children,08-06-1981,M,3.0,France,6
4,51750724947,273420,2019-02-23,6,5,-2,-791,166.110,-1748.110,TeleShop,Books,DIY,27-07-1992,M,8.0,Denmark,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16057,80482431373,269966,2017-01-01,3,2,-2,-277,58.170,-612.170,e-Shop,Footwear,Kids,13-12-1981,M,1.0,Poland,6
16058,66464587778,275091,2017-01-01,8,3,5,94,49.350,519.350,Flagship store,Electronics,Personal Appliances,26-03-1973,F,2.0,Germany,6
16059,76257166854,269407,2017-01-01,8,3,1,112,11.760,123.760,e-Shop,Electronics,Personal Appliances,25-04-1978,F,1.0,Poland,6
16060,78865860112,271275,2017-01-01,3,1,-5,-1344,705.600,-7425.600,TeleShop,Clothing,Kids,01-07-1976,M,8.0,Denmark,6


In [16]:
grouped = df.groupby('tran_date')['total_amt'].sum()
grouped


tran_date
2016-01-25    48957.025
2016-01-26    62045.750
2016-01-27    29257.085
2016-01-28    39730.275
2016-01-29    41354.625
                ...    
2019-02-22    -5965.895
2019-02-23    -3569.150
2019-02-24    -6266.455
2019-02-27    -8270.925
2019-02-28    -4265.300
Name: total_amt, Length: 1128, dtype: float64

In [23]:
app = dash.Dash()

app.layout = html.Div(children=[html.H1('Pogoda we Wrocławiu', style={'text-align': 'center'}),
                                dcc.Graph(id='wro-weather'), dcc.RangeSlider(id='date-slider',
                                                                             min=df.tran_date.min().day,
                                                                             max=df.tran_date.max().day,
                                                                             step=1,
                                                                             marks={date.day: date.strftime(
                                                                                 '%m-%d') for date in df.tran_date.unique()},
                                                                             value=[df.tran_date.min().day, df.tran_date.max().day]),
                                html.Div(id='hover-details', style={'text-align': 'center',
                                                                    'margin-top': '20px',
                                                                    'font-weight': 'bold'})], style={'width': '80%', 'margin': 'auto'})


@app.callback(Output('wro-weather', 'figure'),
              [Input('date-slider', 'value')])
def update_heatmap(x_range):

    wro_temp = df.truncate(before=dt.datetime(2020, 6, x_range[0]),
                           after=dt.datetime(2020, 6, x_range[1])+pd.Timedelta(hours=23))
    fig = tools.make_subplots(rows=1, cols=2, subplot_titles=[
                              'Temperatura', 'Opady'], shared_yaxes=True)

    fig.append_trace(go.Heatmap(x=wro_temp.index.Store_type,
                                y=wro_temp.index.weekday.map({0: 'Poniedziałek',
                                                              1: 'Wtorek',
                                                              2: 'Środa',
                                                              3: 'Czwartek',
                                                              4: 'Piątek',
                                                              5: 'Sobota',
                                                              6: 'Niedziela'}),
                                z=wro_temp['Wrocław Temperature [2 m elevation corrected]'].tolist(
                                ),
                                colorscale='Jet',
                                showscale=False), 1, 1)

    fig.append_trace(go.Heatmap(x=wro_temp.index.hour,
                                y=wro_temp.index.weekday.map({0: 'Poniedziałek',
                                                              1: 'Wtorek',
                                                              2: 'Środa',
                                                              3: 'Czwartek',
                                                              4: 'Piątek',
                                                              5: 'Sobota',
                                                              6: 'Niedziela'}),
                                z=wro_temp['Wrocław Precipitation Total'].tolist(),
                                colorscale='Cividis',
                                showscale=False), 1, 2)

    return fig
@app.callback(Output('hover-details', 'children'),
              [Input('wro-weather', 'hoverData')])
def update_hover_details(hoverData):
    dpoint = hoverData['points'][0]

    if dpoint['curveNumber'] == 0:
        return f"{dpoint['y']}: temperatura powietrza o godzinie {dpoint['x']} wynosiła {round(dpoint['z'],2)}"
    elif dpoint['curveNumber'] == 1:
        return f"{dpoint['y']}: opady o godzinie {dpoint['x']} wynosiły {round(dpoint['z'],2)}"


# if __name__ == '__main__':
#     app.run_server(debug=True, port=8080)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~/anaconda3/lib/python3.11/site-packages/flask/app.py in ?(self=<Flask '__main__'>)
   1818             rv = self.preprocess_request()
   1819             if rv is None:
   1820                 rv = self.dispatch_request()
   1821         except Exception as e:
-> 1822             rv = self.handle_user_exception(e)
   1823         return self.finalize_request(rv)

~/anaconda3/lib/python3.11/site-packages/flask/app.py in ?(self=<Flask '__main__'>)
   1792         ):
   1793             return self.make_default_options_response()
   1794         # otherwise dispatch to the handler for that endpoint
   1795         view_args: t.Dict[str, t.Any] = req.view_args  # type: ignore[assignment]
-> 1796         return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)

~/anaconda3/lib/python3.11/site-packages/dash/dash.py in ?(self=